In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as parquet
import sklearn.metrics
import pickle

In [2]:
#submission = '/home/adeykin/projects/SNAHackaton/out/imagesSubmit.csv'
trainMetaPath = '/home/adeykin/projects/SNAHackaton/imagesTrain'

In [12]:
users = {}
f = open(submission, 'r')
for line in f.readlines():
    userIdStr, postsListStr = line.split(',', 1)
    userId = int(userIdStr)
    users[userId] = list(map(int, postsListStr[2:-3].split(',')))
    break

In [13]:
users

{59: [29096263, 39141251]}

In [3]:
trainMeta = parquet.read_table(trainMetaPath, columns = ['instanceId_userId', 'instanceId_objectId','feedback', 'ImageId']).to_pandas()
trainMeta['ImageId'] = trainMeta['ImageId'].map(lambda x: x[0])
trainMeta['targetScore'] = trainMeta['feedback'].map(lambda x:  (1 if 'Liked' in x else 0) + (-1 if 'Disliked' in x else 0) )
trainMeta.drop(['date','feedback'], axis=1, inplace=True)

In [4]:
trainMeta.head()

,instanceId_userId,instanceId_objectId,ImageId,targetScore
0,116,28780736,1f1614057e4c0c958a90867258fd54d3,0
1,353,26245790,bf0b69e0ed5d2c7e5c819b19267715aa,0
2,707,12497594,45a0714d83500065a5e9a60bf960db58,0
3,1724,16745663,424d49d2a2a6d91d96bc79c7a29dea19,0
4,1730,13138095,6ba59ce7a8298806cd15d9db568f4959,0


In [5]:
trainMeta.to_pickle('trainTargetForROC.pkl')

# Validate

In [3]:
trainMeta = pd.read_pickle('trainTargetForROC.pkl')

In [4]:
prediction = pd.read_pickle('trainValidateOnTrainDF.pkl')

In [5]:
prediction = pd.merge(left=trainMeta, right=prediction, on='ImageId')

In [6]:
prediction.head()

,instanceId_userId,instanceId_objectId,ImageId,targetScore,actualScore
0,116,28780736,1f1614057e4c0c958a90867258fd54d3,0,1.293848
1,1343897,28780736,1f1614057e4c0c958a90867258fd54d3,0,1.293848
2,2819768,28780736,1f1614057e4c0c958a90867258fd54d3,0,1.293848
3,538655,28780736,1f1614057e4c0c958a90867258fd54d3,0,1.293848
4,15147158,28780736,1f1614057e4c0c958a90867258fd54d3,0,1.293848


In [8]:
g = prediction.groupby('instanceId_userId')

In [9]:
k = 0
scores = []
for i, group in g:
    targetScore = group['targetScore'].values
    targetScore[targetScore == -1] = 0
    if targetScore.min() == targetScore.max():
        continue
    actualScore = group['actualScore'].values
    actualScore /= actualScore.max()
    actualScore = 1.0/actualScore
    #print(targetScore)
    #print(actualScore)
    score = sklearn.metrics.roc_auc_score(targetScore, actualScore)
    #print(score)
    scores.append(score)
    #k += 1
    #if k > 100:
    #    break

In [43]:
len(scores)

1256203

In [44]:
np.array(scores).mean()

0.5304517157327377

In [ ]:
#after reversing
np.array(scores).mean()

In [31]:
for user, posts in users.items():
    print(posts)
    print(trainMeta[trainMeta['instanceId_userId']==user])

[29096263, 39141251]
          instanceId_userId  instanceId_objectId  targetScore
6782465                  59             26504003            0
12775945                 59             34971217            1
16198101                 59             11728358            0
16595544                 59             11756551            0


In [33]:
trainActual = pd.read_pickle('trainValidateOnTrainDF.pkl')

In [34]:
trainMeta.head()

,instanceId_userId,instanceId_objectId,targetScore
0,116,28780736,0
1,353,26245790,0
2,707,12497594,0
3,1724,16745663,0
4,1730,13138095,0


In [35]:
trainActual.head()

,ImageId,actualScore
0,000001306cae19a89e295296b506bab7,1.390487
1,000006419b6c05ec9cf30ef466dd8fec,1.304620
2,000007d3372fb4688f870cb150d551f8,1.307641
3,000010bc53c1c6c567c112b7a17227fa,0.805069
4,00001a68648da02922172a45b42f5e16,1.293848
